In [22]:
import pandas as pd
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE 
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, classification_report
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

#### 0 is bad, 1 is good

In [2]:
df = pd.read_csv('after_preprocessing.csv')
df

,text,label,clean
0,I grew up (b. 1965) watching and loving the Th...,0,grew b watching loving thunderbirds mates scho...
1,"When I put this movie in my DVD player, and sa...",0,put movie dvd player sat coke chips expectatio...
2,Why do people who do not know what a particula...,0,people know particular time past like feel nee...
3,Even though I have great interest in Biblical ...,0,even though great interest biblical movies bor...
4,Im a die hard Dads Army fan and nothing will e...,1,im die hard dads army fan nothing ever change ...
...,...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1,western union something forgotten classic west...
39996,This movie is an incredible piece of work. It ...,1,movie incredible piece work explores every noo...
39997,My wife and I watched this movie because we pl...,0,wife watched movie plan visit sicily stromboli...
39998,"When I first watched Flatliners, I was amazed....",1,first watched flatliners amazed necessary feat...


#### Preprocess

In [3]:
def to_lower(text):
    return text.lower()

contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def main_contraction(text):
    text = expand_contractions(text, contractions_dict)
    return text

def remove_numbers(text):
    output = ''.join(c for c in text if not c.isdigit())
    return output

def remove_punctuation(text):
    return ''.join(c for c in text if c not in punctuation)

def to_strip(text):
    return " ".join(text.split())

def remove_stopwords(sentence):
    stop_words = stopwords.words('english')
    return ' '.join([w for w in nltk.word_tokenize(sentence) if not w in stop_words])

In [4]:
def text_preprocessing(sentence):
    prep1 = to_lower(sentence)
    prep2 = main_contraction(prep1)
    prep3 = remove_numbers(prep2)
    prep4 = remove_punctuation(prep3)
    prep5 = to_strip(prep4)
    prep6 = remove_stopwords(prep5)
    
    return prep6

In [5]:
# df['clean']=df['text'].apply(text_preprocessing)

In [6]:
# df

In [7]:
df['label'].value_counts(normalize=True)

0    0.500475
1    0.499525
Name: label, dtype: float64

In [8]:
X = df['clean']
y = df['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=2021)

In [10]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
# smote = SMOTE(random_state=2021)
mnb = MultinomialNB()
# svm = svm.SVC(kernel='linear')
logreg = LogisticRegression(random_state=2021)
# dtc = DecisionTreeClassifier(random_state=2021)

In [12]:
pipeline_mnb = Pipeline([
    ('preprocess',tfidf),
    ('model',mnb)
])

pipeline_logreg = Pipeline([
    ('preprocess',tfidf),
    ('model',logreg)
])

In [13]:
skf = StratifiedKFold(n_splits=3, random_state=2021)

mnb_score = cross_val_score(pipeline_mnb, X_train, y_train, scoring='accuracy', cv=skf, n_jobs=-1, verbose=1)
# svm_score = cross_val_score(pipeline_svm, X_train, y_train, scoring='accuracy', cv=skf, n_jobs=-1, verbose=1)
logreg_score = cross_val_score(pipeline_logreg, X_train, y_train, scoring='accuracy', cv=skf, n_jobs=-1, verbose=1)
# dtc_score = cross_val_score(pipeline_dtc, X_train, y_train, scoring='accuracy', cv=skf, n_jobs=-1, verbose=1)

C:\Users\johan\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   19.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   49.7s finished


In [14]:
print('MNB Score:', mnb_score)
# print('SVM Score:', svm_score)
print('Logreg Score:', logreg_score)
# print('DTC Score:', dtc_score)

MNB Score: [0.88590101 0.87945998 0.87763849]
Logreg Score: [0.88182987 0.87924569 0.87517411]


In [15]:
print('MNB Score:', mnb_score.mean())
# print('SVM Score:', svm_score.mean())
print('Logreg Score:', logreg_score.mean())
# print('DTC Score:', dtc_score.mean())

MNB Score: 0.8809998249577816
Logreg Score: 0.8787498900007394


In [17]:
valid = pd.read_csv('Valid.csv')
valid

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
...,...,...
4995,"Man, I loved this movie! This really takes me ...",1
4996,Recovery is an incredibly moving piece of work...,1
4997,"You can take the crook out of the joint, but i...",1
4998,FUTZ is the only show preserved from the exper...,1


### Multinomial NB

In [16]:
pipeline_mnb.fit(X,y)

Pipeline(steps=[('preprocess', TfidfVectorizer(ngram_range=(1, 2))),
                ('model', MultinomialNB())])

In [25]:
mnb = pipeline_mnb.predict(valid['text'])
valid['mnb']=mnb
print(classification_report(valid['label'],valid['mnb']))

              precision    recall  f1-score   support

           0       0.95      0.62      0.75      2486
           1       0.72      0.97      0.83      2514

    accuracy                           0.80      5000
   macro avg       0.84      0.79      0.79      5000
weighted avg       0.83      0.80      0.79      5000



### Logistic Regression

In [26]:
pipeline_logreg.fit(X,y)

Pipeline(steps=[('preprocess', TfidfVectorizer(ngram_range=(1, 2))),
                ('model', LogisticRegression(random_state=2021))])

In [27]:
logreg = pipeline_logreg.predict(valid['text'])
valid['logreg']=logreg
print(classification_report(valid['label'],valid['logreg']))

              precision    recall  f1-score   support

           0       0.94      0.77      0.85      2486
           1       0.81      0.95      0.87      2514

    accuracy                           0.86      5000
   macro avg       0.87      0.86      0.86      5000
weighted avg       0.87      0.86      0.86      5000



### I will take logistic regression as the better model

In [28]:
import joblib

In [29]:
joblib.dump(pipeline_logreg,'sentiment_analysis')

['sentiment_analysis']

In [36]:
coba = pd.Series('Looks like the flower is pretty')

In [37]:
pipeline_logreg.predict(coba)

array([0], dtype=int64)